In [ ]:
# 데이터 가져 오기
import requests
import os
from dotenv import load_dotenv

# .env 파일의 환경 변수를 로드
load_dotenv()

# .env 파일에서 변수 가져오기
master_key = os.getenv('MASTER_KEY')

# JSON 데이터를 가져올 URL
url = 'https://www.meilisearch.com/movies.json'

# HTTP GET 요청으로 JSON 데이터 가져오기
response = requests.get(url)

# 응답이 성공적인지 확인
if response.status_code == 200:
    # JSON 데이터를 로컬 파일로 저장
    with open('./downloads/movies.json', 'wb') as file:
        file.write(response.content)
    print("파일이 성공적으로 저장되었습니다.")
else:
    print(f"데이터를 가져오는 데 실패했습니다. 상태 코드: {response.status_code}")

# 검색엔진에 넣기
import meilisearch
client = meilisearch.Client('http://localhost:7700', master_key)
# 인덱스가 존재하면 삭제
client.index('movies').delete()
print('index deleted')
client.create_index('movies', {'primaryKey': 'id'})
print(f"movies created successfully")
client.index('movies').add_documents(response.content)

In [ ]:
########### 데이터셋 중 '걸그룹' 이란 단어가 포함된 문서만을 파일로 저장

#from datasets import load_dataset
#dataset = load_dataset("heegyu/namuwiki", split='train[:90%]')
#filtered = dataset.filter(lambda x: len(x['text']) < 3000).filter(lambda x: '걸그룹' in x['text'])
#print(len(filtered))
#filtered.to_json("./downloads/girlgroup.ndjson")

########### 파일을 읽어서 간단한 전처리 수행합니다.
import jsonlines, json
import re
def mediaWiki_to_markdown(text):
    # 헤더 변환
    text = re.sub(r'(={2,})(.*?)\1', lambda match: f"{'#' * len(match.group(1))} {match.group(2)}", text)
    # 굵은 글씨 변환
    text = re.sub(r"'''(.*?)'''", r"**\1**", text)
    # 기울임 변환
    text = re.sub(r"''(.*?)''", r"*\1*", text)
    # 링크 변환
    text = re.sub(r'\[\[(.*?)\]\]', r'[\1](\1)', text)
    text = re.sub(r'\[\[(.*?)\|(.*?)\]\]', r'[\2](\1)', text)
    # 이미지 변환
    text = re.sub(r'\[\[파일:(.*?)\|.*?\]\]', r'![\1](\1)', text)
    # 목록 변환
    text = re.sub(r'^\* (.*)', r'* \1', text, flags=re.MULTILINE)
    # 인용 변환
    text = re.sub(r'^>(.*)', r'> \1', text, flags=re.MULTILINE)
    return text

documents = []
with jsonlines.open("./downloads/girlgroup.ndjson") as f:
  with open('./downloads/girlgroup.json', 'w') as f_out:
    f_out.write('[\n')
    for i, line in enumerate(f.iter()):
      # 너무 긴 문서는 제외
      if len(line["text"]) > 3000:
        continue
      line["id"] = i
      line["text"] = mediaWiki_to_markdown(line["text"])
      documents.append(line)
      if i > 0: f_out.write(',')
      f_out.write(json.dumps(line))
      f_out.write('\n')
    f_out.write(']')

docs = documents
# 검색엔진에 넣습니다. python sdk 사용
import meilisearch
import os
from dotenv import load_dotenv

# .env 파일의 환경 변수를 로드
load_dotenv()

# .env 파일에서 변수 가져오기
master_key = os.getenv('MASTER_KEY')

client = meilisearch.Client('http://localhost:7700', master_key)
# 인덱스가 존재하면 삭제
client.index('girlgroup').delete()
print('index deleted')
client.create_index('girlgroup', {'primaryKey': 'id'})
print(f"girlgroup created successfully")
client.index('girlgroup').add_documents(docs)

In [ ]:
########### 파일을 읽어서 간단한 전처리 수행합니다.
import json

with open ("./downloads/WARW1800000007.json", "r") as f:
    json_data = json.load(f)
    paragraph = json_data['document'][0]['paragraph']

with open('./downloads/test.json', "w") as file:
    file.write(json.dumps(paragraph))

documents = []
with open("./downloads/test.json", "r") as f:
    target_json = json.load(f)
    print(len(target_json))
    for i in range(len(target_json)):
      target_json[i]["id"] = i
      target_json[i]["form"] = target_json[i]["form"]
      documents.append(target_json[i])

docs = documents

# 검색엔진에 넣습니다. python sdk 사용
import meilisearch
import os
from dotenv import load_dotenv

# .env 파일의 환경 변수를 로드
load_dotenv()

# .env 파일에서 변수 가져오기
master_key = os.getenv('MASTER_KEY')

client = meilisearch.Client('http://localhost:7700', master_key)
# 인덱스가 존재하면 삭제
client.index('WARW1800000007').delete()
print('index WARW1800000007 deleted')
client.create_index('WARW1800000007', {'primaryKey': 'id'})
print(f"WARW1800000007 created successfully")
client.index('WARW1800000007').add_documents(docs)